In [1]:
import os
import json

import pandas as pd
import pyarrow as pa
from pyarrow import parquet as pq

In [2]:
# Function to read parquet file
def read_parquet_file(file_path: str) -> pa.Table:
    table = pq.read_table(file_path)
    return table

def read_jsonl(file_path: str) -> pa.Table:
    with open(file_path, "r") as file:
        data = [json.loads(line)['text'] for line in file]
    df = pd.DataFrame(data, columns=['text'])
    table = pa.Table.from_pandas(df)
    return table

def read_csv(file_path: str) -> pa.Table:
    df = pd.read_csv(file_path)
    table = pa.Table.from_pandas(df)
    return table

In [3]:
os.getcwd()

'/workspace'

In [4]:
os.chdir('/workspace/koko/myllm')
os.getcwd()

'/workspace/koko/myllm'

# Load all data

In [5]:
paths = [
    "datasets/tokenizer/raw", 
    'datasets/pretrain/computer_programming_code/chinese/high', 
    'datasets/pretrain/computer_programming_code/chinese/middle',
    'datasets/pretrain/computer_programming_code/english/high', 
    'datasets/pretrain/computer_programming_code/english/middle',
    'datasets/pretrain/artificial_intelligence_machine_learning/english/high', 
    'datasets/pretrain/artificial_intelligence_machine_learning/english/middle',
    'datasets/pretrain/artificial_intelligence_machine_learning/chinese/high',
    'datasets/pretrain/artificial_intelligence_machine_learning/chinese/middle'
]

In [6]:
# Get all files in the directories
files = []

for path in paths:
    fs = os.listdir(path)
    fs = [os.path.join(path, f) for f in fs]
    files.extend(fs)
len(files), files[:5]

(19,
 ['datasets/tokenizer/raw/minimind_tokenizer_train.jsonl',
  'datasets/pretrain/computer_programming_code/chinese/high/rank_00000_00000.parquet',
  'datasets/pretrain/computer_programming_code/chinese/middle/rank_00000_00000.parquet',
  'datasets/pretrain/computer_programming_code/english/high/rank_00000_00001.parquet',
  'datasets/pretrain/computer_programming_code/english/high/rank_00000_00000.parquet'])

In [7]:
# Read all files
data = []
for file in files:
    if file.endswith(".parquet"):
        data.append(read_parquet_file(file))
    elif file.endswith(".json"):
        data.append(read_jsonl(file))
    else:
        print(f"Unknown file type: {file}")

Unknown file type: datasets/tokenizer/raw/minimind_tokenizer_train.jsonl


In [11]:
data[1]

pyarrow.Table
text: string
alnum_ratio: double
avg_line_length: double
char_rep_ratio: double
flagged_words_ratio: double
max_line_length: int64
num_words: int64
perplexity: double
quality_score: double
special_char_ratio: double
word_rep_ratio: double
_id: int64
industry_type: string
----
text: [["浮点型(Float) Python的浮点数就是数学中的小数,类似C语言中的double. 在运算中,整数与浮点数运算的结果是浮点数. 浮点数也就是小数,之所以称为浮点数,是因为按照科学记数法表示时,一个浮点数的小数点位置是可变的,比如,1.23x109和12.3x108是相等的.浮点数可以用数学写法,如1.23,3.14,-9.01,等等.但是对于很大或很小的浮点数,就必须用科学计数法表示,把10用e替代,1.23x109就是1.23e9,或者12.3e8,0.000012可以写成1.2e-5,等等. 整数和浮点数在计算机内部存储的方式是不同的,整数运算永远是精确的而浮点数运算则可能会有四舍五入的误差.
影子曹痴梅一点%门锁狗坏#python浮点数的意思:python中整数是int类型的,浮点数是float类型的,浮点数也就是小数示例代码如下:执行结果如下:更多Python知识,请关注:Python自学网!","罗马不是一天建成的,编程语言亦是如此,每一门编程语言在借鉴前辈的同时,也会提出自己独有的特性,Rust 即是如此.当站在巨人肩膀上时,一个人所能看到的就更高更远,恰好,我们看到了函数式语言的优秀特性,例如:
见猎心喜,我们忍不住就借鉴了过来,于是你能看到本章的内容,天下语言一大.跑题了.
关于函数式编程到底是什么的争论由来已久,本章节并不会踏足这个泥潭,因此我们在这里主要关注的是函数式特性:
其中后两个在前面章节我们已经深入学习过,因此本章的重点就是闭包和迭代器,这些函数式特性可以让代码的可读性和易写性大幅提升.对于 Rust 语言来说,掌握这两者就相当于你同时拥有了倚

In [12]:
# Traverse all the data and drop all the columns except the text column
for i in range(len(data)):
    data[i] = data[i].select(['text'])

In [19]:
# Concatenate all the tables
data = pa.concat_tables(data)
data

pyarrow.Table
text: string
----
text: [["放心这是个好事,因为这让 CUE 更好理解而且能表示更多东西.这让我们能在同一个文件中定义类型,进行约束和生成数据,也意味着定义类型对人类思考然后实现它都将变得更容易.
虽然你可能更熟悉类型(type)和数据(data),大多数语言不会把约束条件(constraint)或者校验(validation)作为首要概念. 但是 CUE 恰恰相反,用 约束条件(constraint)来管理值的规则或限制,像其他所有的实体一样,它存在于结构和具体数值中间.
CUE 的最佳实践让我们从范围比较大的结构开始,然后逐渐限制,一直到具体的实体结束.
一开始都是从很小的结构定义,然后最终将它们改造成复杂的实体.
要理解 CUE 最重要的概念之一是 数值不能被概念,在 CUE 中没有重载或重写的概念.
写代码或组织代码的方式都会影响值的变化,这样做的原因是为了维护性和可理解,同时这也是 CUE 的哲学所要求的.
当你在你的配置中找数值在哪里设置的时候,你就会发现其实这很有用.
CUE 不仅仅确保那个值是你设置的,而且会告诉你在哪里设置的,如果发生冲突,也会告诉你冲突的地方.
CUE 允许字段被定义多次,只要保持一致的就没问题.
使用这些属性对于在一个地方定义结构,然后在另一个地方定义约束,构建配置 以及确保一个字段没有在不同地方错误地设置为不同的值很有用.
definition 是 CUE 定义结构的方法,它和 struct 有些不同的规则.
CUE 支持设置值的默认值或指定字段是可选的.
没有包含所有字段的数据,可以成为不完全的值,CUE 将不会导出未完成的值,而会返回错误.
相反的,完全的值是那些所有字段都被指定的数据.
开放表示 struct 可以被扩展,而封闭表示不能被扩展.
默认情况下,struct 是开放的,而 definition 是封闭的.
CUE 也支持让我们显式地做相反的声明.
在 CUE 中,推荐从很小的定义开始逐渐构建值,让一些结构定义能重用,可以通过嵌套实现定义.
CUE 的合并系统涉及到值,结构和约束条件,但不会考虑顺序或者在哪个文件中.
简单的说,合并只是组合,交换,而且是幂等的.
CUE 是图灵非完备的,意味着不像是通常意义的语言.你只提供值,类型,定义和约束条件,然后 CU

In [36]:
data['text'][0].as_py()

'放心这是个好事,因为这让 CUE 更好理解而且能表示更多东西.这让我们能在同一个文件中定义类型,进行约束和生成数据,也意味着定义类型对人类思考然后实现它都将变得更容易.\n虽然你可能更熟悉类型(type)和数据(data),大多数语言不会把约束条件(constraint)或者校验(validation)作为首要概念. 但是 CUE 恰恰相反,用 约束条件(constraint)来管理值的规则或限制,像其他所有的实体一样,它存在于结构和具体数值中间.\nCUE 的最佳实践让我们从范围比较大的结构开始,然后逐渐限制,一直到具体的实体结束.\n一开始都是从很小的结构定义,然后最终将它们改造成复杂的实体.\n要理解 CUE 最重要的概念之一是 数值不能被概念,在 CUE 中没有重载或重写的概念.\n写代码或组织代码的方式都会影响值的变化,这样做的原因是为了维护性和可理解,同时这也是 CUE 的哲学所要求的.\n当你在你的配置中找数值在哪里设置的时候,你就会发现其实这很有用.\nCUE 不仅仅确保那个值是你设置的,而且会告诉你在哪里设置的,如果发生冲突,也会告诉你冲突的地方.\nCUE 允许字段被定义多次,只要保持一致的就没问题.\n使用这些属性对于在一个地方定义结构,然后在另一个地方定义约束,构建配置 以及确保一个字段没有在不同地方错误地设置为不同的值很有用.\ndefinition 是 CUE 定义结构的方法,它和 struct 有些不同的规则.\nCUE 支持设置值的默认值或指定字段是可选的.\n没有包含所有字段的数据,可以成为不完全的值,CUE 将不会导出未完成的值,而会返回错误.\n相反的,完全的值是那些所有字段都被指定的数据.\n开放表示 struct 可以被扩展,而封闭表示不能被扩展.\n默认情况下,struct 是开放的,而 definition 是封闭的.\nCUE 也支持让我们显式地做相反的声明.\n在 CUE 中,推荐从很小的定义开始逐渐构建值,让一些结构定义能重用,可以通过嵌套实现定义.\nCUE 的合并系统涉及到值,结构和约束条件,但不会考虑顺序或者在哪个文件中.\n简单的说,合并只是组合,交换,而且是幂等的.\nCUE 是图灵非完备的,意味着不像是通常意义的语言.你只提供值,类型,定义和约束条件,然后 CUE 会告诉你写的是不是正确. 这是有意为

In [35]:
print(data.column_names)

['text']


In [28]:
data.shape

(2677127, 1)

In [20]:
# Save the data
pq.write_table(data, 'datasets/pretrain/pretrain.parquet')